In [ ]:
import os
os.chdir(os.path.expanduser("/Users/barkinkaradeniz/Projects/ChatGPTStockPredicter copy/"))

from src.mongo.pymongo_get_database import get_database

In [ ]:
import pandas as pd

dbname = get_database()
appleChanges = dbname["AppleDailyStockData"]
appleSentimentAverages = dbname["AppleSentimentAverages"]

df = pd.DataFrame(list(appleChanges.find()))

df["Sentiment"] = [average["sentimentAverage"] for average in appleSentimentAverages.find()]

df

In [ ]:
df = df[['Date', 'Close', 'Sentiment']]

df

In [ ]:
df['Date']

In [ ]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

In [ ]:
df

In [ ]:
df['Date'] = df['Date'].apply(str_to_datetime)
df['Close'] = df['Close'].astype(float)
df['Sentiment'] = df['Sentiment'].astype(float)

df['Date']

In [ ]:
df.index = df.pop('Date')
df

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df.index, df['Close'])

In [ ]:
plt.plot(df.index, df['Sentiment'])

In [ ]:
import numpy as np

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3, m=5):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

##############################################################################################################  

  target_date = first_date

  X = []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(m)

    if len(df_subset) != m:
      print(f'Error: Window of size {m} is too large for date {target_date}')
      return

    values = df_subset['Sentiment'].to_numpy()

    X.append(values)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  X = np.array(X)
  for i in range(0, m):
    X[:, i]
    ret_df[f'Sentiment-{m-i-1}'] = X[:, i]

##############################################################################################################

  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df,
                                '2022-03-14',
                                '2023-07-21',
                                n=5,
                                m=6)

windowed_df

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scX = StandardScaler()
# scY = StandardScaler()

# windowed_df.loc[:, windowed_df.columns[1:-1]] = scX.fit_transform(windowed_df.loc[:, windowed_df.columns[1:-1]])
# windowed_df.loc[:, windowed_df.columns[-1]] = scY.fit_transform(windowed_df.loc[:, windowed_df.columns[-1]].values.reshape(-1, 1))

# windowed_df

In [ ]:
def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)

dates.shape, X.shape, y.shape

In [ ]:
q_80 = int(len(dates) * .8)
q_90 = int(len(dates) * .9)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

plt.legend(['Train', 'Validation', 'Test'])

In [ ]:
# from tensorboard.plugins.hparams import api as hp
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import layers
# import tensorflow as tf
# # from helpers import *

# HP_NUM_UNITS_LSTM = hp.HParam('Number of Hidden Neurons LSTM', hp.Discrete([32,64,128]))
# HP_NUM_UNITS = hp.HParam('Number of Hidden Neurons', hp.Discrete([16,32,64]))
# HP_LEARNING_RATE = hp.HParam('Learning Rate', hp.Discrete([0.0001, 0.001, 0.01]))
# HP_BATCH_SIZE = hp.HParam('Batch Size', hp.Discrete([32, 64, 128]))
# HP_NUM_LAYERS = hp.HParam('Number of Hidden Layers', hp.Discrete([2, 3]))
# HP_NUM_EPOCHS = hp.HParam('Number of Epochs,', hp.Discrete([60, 100, 120]))

# #Setting the Metric to MSE
# METRIC_MSE = 'Validation Loss (MSE)'

# #Creating & configuring log files
# with tf.summary.create_file_writer('logs/HPT_secondStage').as_default():
#   hp.hparams_config(
#     hparams=[HP_NUM_UNITS_LSTM, HP_NUM_UNITS, HP_LEARNING_RATE, HP_BATCH_SIZE, HP_NUM_LAYERS, HP_NUM_EPOCHS],
#     metrics=[hp.Metric(METRIC_MSE, display_name='Validation Loss (MSE)')]
#   )

# def train_test_model(hparams):
#   model = tf.keras.models.Sequential()

#   model.add(tf.keras.layers.Input((11, 1)))

#   model.add(tf.keras.layers.LSTM(hparams[HP_NUM_UNITS_LSTM]))

#   for _ in range(hparams[HP_NUM_LAYERS]):
#     model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu")) 

#   model.add(tf.keras.layers.Dense(1))

#   model.compile(
#       optimizer=Adam(learning_rate=hparams[HP_LEARNING_RATE]),
#       loss='mse',
#       metrics=['mean_absolute_error'],
#   )

#   history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=hparams[HP_NUM_EPOCHS], batch_size=hparams[HP_BATCH_SIZE])
#   best_mse = float('inf')
#   for val_loss in history.history['val_loss']:
#     if val_loss < best_mse:
#       best_mse = val_loss
#   print(best_mse)
#   return best_mse

# def run(run_dir, hparams):
#   with tf.summary.create_file_writer(run_dir).as_default():
#       hp.hparams(hparams)  
#       mse = train_test_model(hparams)
#       tf.summary.scalar(METRIC_MSE, mse, step=1)

# session_num = 00

# for num_units_lstm in HP_NUM_UNITS_LSTM.domain.values:
#     for num_units in HP_NUM_UNITS.domain.values:
#         for learning_rate in HP_LEARNING_RATE.domain.values:
#             for batch_size in HP_BATCH_SIZE.domain.values:
#                 for num_layers in HP_NUM_LAYERS.domain.values:
#                     for num_epochs in HP_NUM_EPOCHS.domain.values:
#                         hparams = {
#                             HP_NUM_UNITS_LSTM: num_units_lstm,
#                             HP_NUM_UNITS: num_units,
#                             HP_LEARNING_RATE: learning_rate,
#                             HP_BATCH_SIZE: batch_size,
#                             HP_NUM_LAYERS: num_layers,
#                             HP_NUM_EPOCHS: num_epochs
#                         }

#                         run_name = "run-%d" % session_num
#                         print('--- Starting trial: %s' % run_name)
#                         print({h.name: hparams[h] for h in hparams})
#                         run('logs/HPT_secondStage/Apple/' + run_name, hparams)
#                         session_num += 1

# # cmd+shift+p

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

model = Sequential([layers.Input((11, 1)),
                    layers.LSTM(128),
                    layers.Dense(64, activation='relu'),
                    layers.Dense(64, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.01),
              metrics=['mean_absolute_error'])

results = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

In [ ]:
plt.plot(results.history["loss"])
plt.plot(results.history["val_loss"])
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend(['Training Loss', 'Validation Loss'])

In [ ]:
train_predictions = model.predict(X_train).flatten()
val_predictions = model.predict(X_val).flatten()
test_predictions = model.predict(X_test).flatten()

# y_train = scY.inverse_transform(y_train.reshape(-1, 1)).flatten()
# train_predictions = scY.inverse_transform(train_predictions.reshape(-1, 1)).flatten()
# y_val = scY.inverse_transform(y_val.reshape(-1, 1)).flatten()
# val_predictions = scY.inverse_transform(val_predictions.reshape(-1, 1)).flatten()
# y_test = scY.inverse_transform(y_test.reshape(-1, 1)).flatten()
# test_predictions = scY.inverse_transform(test_predictions.reshape(-1, 1)).flatten()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

print('Test Mean Absolute Percentage Error:', mean_absolute_percentage_error(y_test, test_predictions))
print(f'Test Mean Absolute Percentage Error: {np.mean(np.abs((y_test - test_predictions) / y_test)) * 100}')

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend(['Training Predictions', 'Training Observations'])

In [ ]:
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend(['Validation Predictions', 'Validation Observations'])

In [ ]:
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend(['Testing Predictions', 'Testing Observations'])

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])

In [ ]:
last_value_y = y_val[-1]

y_test_binary = np.zeros_like(y_test)
test_predictions_binary = np.zeros_like(test_predictions)

for i in range(len(y_test)):
    if y_test[i] > last_value_y:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

    if test_predictions[i] > last_value_y:
        test_predictions_binary[i] = 1
    else:
        test_predictions_binary[i] = 0

    last_value_y = y_test[i]

print(np.concatenate((y_test_binary.reshape(len(y_test),1), test_predictions_binary.reshape(len(test_predictions_binary),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test_binary, test_predictions_binary)
print(cm)
accuracy_score(y_test_binary, test_predictions_binary)